### Text file, here english wikipedia

In [9]:
import torch

In [10]:
import torch.utils.data as data

In [11]:
from ipywidgets import FloatProgress
from IPython.display import display

In [12]:
wiki_file = "/data/datasets/enwiki_pre.txt"

In [13]:
def countlines(file):
    c = 0
    for line in file:
        c += 1
    file.seek(0)
    return c

In [44]:
def createDictionnary(text_file, min_count=30):
    word2id = {}
    wordCount = {}
    i = 0
    with open(text_file,encoding="ISO-8859-1") as f:
        c = countlines(f)
        prog = FloatProgress(min=0, max=c)
        display(prog)
        for j, line in enumerate(f):
            if j % 1000 == 0:
                prog.value += 1000
            #update_progress(j / c)
            #print("%2.2f" %(j/c*100), end='')            
            for word in line.split(' '):
                if not word in word2id:
                    word2id[word] = i
                    i += 1
                
                if not word in wordCount:
                    wordCount[word] = 1
                else:
                    wordCount[word] += 1
    word2id = {w:word2id[w] for w in word2id if wordCount[w] >= min_count}
    print("Dictionnary of ", len(word2id), "words")
    return word2id

In [45]:
class contextDataset(data.Dataset):
    def __init__(self, text_file, word2id=None):
        if word2id is None:
            self.word2id = createDictionnary(text_file)
        self.text = []
        with open(text_file, encoding="ISO-8859-1") as f:
            print("Reading text file")
            prog = FloatProgress(min=0, max=countlines(f))
            for i, line in enumerate(f):
                if i%1000 == 0:
                    prog.value += 1000
                self.text.extend([w for w in line.split(' ') if w in self.word2id])
            
        self.word2id = word2id
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, i):
        target = torch.LongTensor([self.word2id[self.text[i]]])
        if i >= 2 and i < len(text) - 2:
            context = [self.text[i - 2], self.text[i - 1],
                      self.text[i + 1], self.text[i + 2]]
        elif i == 1:
            context = [self.text[i - 1], self.text[i + 1],
                      self.text[i + 2], self.text[i + 3]]
        elif i == 0:
            context = [self.text[i + 1], self.text[i + 2], 
                       self.text[i + 3], self.text[i+4]]
        elif i >= len(text) - 2:
            context = [self.text[i-4], self.text[i-3] ,self.text[i - 2], self.text[i - 1]]
        else:
            print("ERROOR :", i, '/', len(self.text))
            return None
        context = [self.word2id[c] for c in context]
        
        neg = random.choices(list(word2id.values()), k=1)
        
        return target, context, neg

In [46]:
cd = contextDataset(wiki_file)

FloatProgress(value=0.0, max=85778920.0)

Dictionnary of  578642 words
Reading text file


In [43]:
print(cd.word2id)

None
